In [1]:
import xarray as xr
import numpy as np
import pandas as pd
np.random.seed(0)

# Working with labeled data

## Named dimensions

As mentioned in the previous session, labeled dimensions really help to make the code less difficult to understand. Compare pure `numpy` indexing:

In [2]:
# axis0: x, axis1: y
np_array = np.random.randn(3, 4)
np_array[1, 3]

-0.1513572082976979

and slicing:

In [3]:
np_array[:2, 1:]

array([[ 0.40015721,  0.97873798,  2.2408932 ],
       [-0.97727788,  0.95008842, -0.15135721]])

with label based indexing:

In [4]:
arr = xr.DataArray(np_array, dims=("x", "y"))
arr.isel(x=1, y=3)

<xarray.DataArray ()>
array(-0.15135721)

This is the same as

In [5]:
arr[{"x": 1, "y": 1}]

<xarray.DataArray ()>
array(-0.97727788)

Due to the language syntax, slices have to be constructed manually:

In [6]:
ds = xr.Dataset({
    "a": (("x", "y"), np.random.randn(3, 4)),
    "b": (("x", "y"), np.random.randn(3, 4)),
})
ds.isel(x=slice(None, 2), y=slice(1, None))

<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Dimensions without coordinates: x, y
Data variables:
    a        (x, y) float64 0.1217 0.4439 0.3337 -0.2052 0.3131 -0.8541
    b        (x, y) float64 -1.454 0.04576 -0.1872 1.469 0.1549 0.3782

We can also use these names to peek at the data if the automatic preview is not enough:

In [7]:
ds.head(x=2, y=3)

<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Dimensions without coordinates: x, y
Data variables:
    a        (x, y) float64 0.761 0.1217 0.4439 1.494 -0.2052 0.3131
    b        (x, y) float64 2.27 -1.454 0.04576 1.533 1.469 0.1549

see also `tail` and `thin`.

## Coordinate labels and label based indexing

xarray objects become much more interesting when adding coordinate labels:

In [8]:
arr = xr.DataArray(
    np.random.randn(4, 6),
    dims=("x", "y"),
    coords={
        "x": [-3.2, 2.1, 5.3, 6.5],
        "y": pd.date_range("2009-01-05", periods=6, freq="M"),
    },
)
arr

<xarray.DataArray (x: 4, y: 6)>
array([[ 1.23029068,  1.20237985, -0.38732682, -0.30230275, -1.04855297,
        -1.42001794],
       [-1.70627019,  1.9507754 , -0.50965218, -0.4380743 , -1.25279536,
         0.77749036],
       [-1.61389785, -0.21274028, -0.89546656,  0.3869025 , -0.51080514,
        -1.18063218],
       [-0.02818223,  0.42833187,  0.06651722,  0.3024719 , -0.63432209,
        -0.36274117]])
Coordinates:
  * x        (x) float64 -3.2 2.1 5.3 6.5
  * y        (y) datetime64[ns] 2009-01-31 2009-02-28 ... 2009-05-31 2009-06-30

To select data by coordinate labels instead of integer indices we can use the same syntax, using `sel` instead of `isel`:

In [9]:
arr.sel(x=5.3, y="2009-04-30")  # or a.loc[{"x": 5.3, "y": "2009-04-30"}]

<xarray.DataArray ()>
array(0.3869025)
Coordinates:
    x        float64 5.3
    y        datetime64[ns] 2009-04-30

this will require us to specify exact values. If we don't have those, we can use the `method` parameter (see `Dataset.sel` for documentation):

In [10]:
arr.sel(x=4, y="2009-04-01", method="nearest")

<xarray.DataArray ()>
array(-0.89546656)
Coordinates:
    x        float64 5.3
    y        datetime64[ns] 2009-03-31

We can also select multiple values:

In [11]:
arr.sel(x=[-3.2, 6.5], y=slice("2009-02-28", "2009-05-31"))

<xarray.DataArray (x: 2, y: 4)>
array([[ 1.20237985, -0.38732682, -0.30230275, -1.04855297],
       [ 0.42833187,  0.06651722,  0.3024719 , -0.63432209]])
Coordinates:
  * x        (x) float64 -3.2 6.5
  * y        (y) datetime64[ns] 2009-02-28 2009-03-31 2009-04-30 2009-05-31

If instead of selecting data we want to drop it, we can use `drop_sel`:

In [12]:
arr.drop_sel(x=[-3.2, 5.3])

<xarray.DataArray (x: 2, y: 6)>
array([[-1.70627019,  1.9507754 , -0.50965218, -0.4380743 , -1.25279536,
         0.77749036],
       [-0.02818223,  0.42833187,  0.06651722,  0.3024719 , -0.63432209,
        -0.36274117]])
Coordinates:
  * x        (x) float64 2.1 6.5
  * y        (y) datetime64[ns] 2009-01-31 2009-02-28 ... 2009-05-31 2009-06-30

## Interpolation

If we want to look at values between the current grid cells (interpolation), we can do that with `interp` (requires `scipy`):

In [13]:
arr.interp(
    x=np.linspace(2, 6, 10),
    y=pd.date_range("2009-04-01", "2009-04-30", freq="D"),
)

<xarray.DataArray (x: 10, y: 30)>
array([[-0.50494977, -0.50255538, -0.500161  , -0.49776661, -0.49537223,
        -0.49297784, -0.49058345, -0.48818907, -0.48579468, -0.4834003 ,
        -0.48100591, -0.47861152, -0.47621714, -0.47382275, -0.47142837,
        -0.46903398, -0.46663959, -0.46424521, -0.46185082, -0.45945643,
        -0.45706205, -0.45466766, -0.45227328, -0.44987889, -0.4474845 ,
        -0.44509012, -0.44269573, -0.44030135, -0.43790696, -0.43551257],
       [-0.54445061, -0.53772041, -0.5309902 , -0.52426   , -0.5175298 ,
        -0.51079959, -0.50406939, -0.49733919, -0.49060898, -0.48387878,
        -0.47714858, -0.47041837, -0.46368817, -0.45695797, -0.45022776,
        -0.44349756, -0.43676736, -0.43003715, -0.42330695, -0.41657675,
        -0.40984654, -0.40311634, -0.39638614, -0.38965594, -0.38292573,
        -0.37619553, -0.36946533, -0.36273512, -0.35600492, -0.34927472],
       [-0.59243043, -0.58009471, -0.56775899, -0.55542327, -0.54308755,
        -0.53075184, -0.51841612, -0.5060804 , -0.49374468, -0.48140896,
        -0.46907325, -0.45673753, -0.44440181, -0.43206609, -0.41973037,
        -0.40739466, -0.39505894, -0.38272322, -0.3703875 , -0.35805178,
        -0.34571607, -0.33338035, -0.32104463, -0.30870891, -0.29637319,
        -0.28403748, -0.27170176, -0.25936604, -0.24703032, -0.2346946 ],
       [-0.64041024, -0.62246901, -0.60452778, -0.58658654, -0.56864531,
        -0.55070408, -0.53276285, -0.51482161, -0.49688038, -0.47893915,
        -0.46099791, -0.44305668, -0.42511545, -0.40717422, -0.38923298,
        -0.37129175, -0.35335052, -0.33540929, -0.31746805, -0.29952682,
        -0.28158559, -0.26364435, -0.24570312, -0.22776189, -0.20982066,
        -0.19187942, -0.17393819, -0.15599696, -0.13805573, -0.12011449],
       [-0.68839006, -0.66484331, -0.64129656, -0.61774981, -0.59420307,
        -0.57065632, -0.54710957, -0.52356283, -0.50001608, -0.47646933,
        -0.45292258, -0.42937584, -0.40582909, -0.38228234, -0.35873559,
        -0.33518885, -0.3116421 , -0.28809535, -0.2645486 , -0.24100186,
        -0.21745511, -0.19390836, -0.17036161, -0.14681487, -0.12326812,
        -0.09972137, -0.07617462, -0.05262788, -0.02908113, -0.00553438],
       [-0.73636987, -0.70721761, -0.67806535, -0.64891309, -0.61976082,
        -0.59060856, -0.5614563 , -0.53230404, -0.50315178, -0.47399951,
        -0.44484725, -0.41569499, -0.38654273, -0.35739047, -0.3282382 ,
        -0.29908594, -0.26993368, -0.24078142, -0.21162915, -0.18247689,
        -0.15332463, -0.12417237, -0.09502011, -0.06586784, -0.03671558,
        -0.00756332,  0.02158894,  0.0507412 ,  0.07989347,  0.10904573],
       [-0.78434969, -0.74959191, -0.71483413, -0.68007636, -0.64531858,
        -0.6105608 , -0.57580303, -0.54104525, -0.50628747, -0.4715297 ,
        -0.43677192, -0.40201414, -0.36725637, -0.33249859, -0.29774081,
        -0.26298304, -0.22822526, -0.19346748, -0.15870971, -0.12395193,
        -0.08919415, -0.05443638, -0.0196786 ,  0.01507918,  0.04983696,
         0.08459473,  0.11935251,  0.15411029,  0.18886806,  0.22362584],
       [-0.8323295 , -0.79196621, -0.75160292, -0.71123963, -0.67087634,
        -0.63051305, -0.59014975, -0.54978646, -0.50942317, -0.46905988,
        -0.42869659, -0.3883333 , -0.34797001, -0.30760671, -0.26724342,
        -0.22688013, -0.18651684, -0.14615355, -0.10579026, -0.06542696,
        -0.02506367,  0.01529962,  0.05566291,  0.0960262 ,  0.13638949,
         0.17675278,  0.21711608,  0.25747937,  0.29784266,  0.33820595],
       [-0.65528226, -0.61996487, -0.58464749, -0.5493301 , -0.51401272,
        -0.47869533, -0.44337795, -0.40806056, -0.37274318, -0.33742579,
        -0.30210841, -0.26679102, -0.23147364, -0.19615625, -0.16083887,
        -0.12552148, -0.0902041 , -0.05488671, -0.01956933,  0.01574806,
         0.05106544,  0.08638283,  0.12170021,  0.1570176 ,  0.19233498,
         0.22765237,  0.26296975,  0.29828714,  0.33360452,  0.36892191],
       [-0.3119

when trying to extrapolate, the resulting values will be `nan`.

If we already have a object with the desired coordinates, we can use `interp_like`:

In [14]:
other = xr.DataArray(
    dims=("x", "y"),
    coords={
        "x": np.linspace(2, 4, 10),
        "y": pd.date_range("2009-04-01", "2009-04-30", freq="D"),
    },
)
arr.interp_like(other)

<xarray.DataArray (x: 10, y: 30)>
array([[-0.50494977, -0.50255538, -0.500161  , -0.49776661, -0.49537223,
        -0.49297784, -0.49058345, -0.48818907, -0.48579468, -0.4834003 ,
        -0.48100591, -0.47861152, -0.47621714, -0.47382275, -0.47142837,
        -0.46903398, -0.46663959, -0.46424521, -0.46185082, -0.45945643,
        -0.45706205, -0.45466766, -0.45227328, -0.44987889, -0.4474845 ,
        -0.44509012, -0.44269573, -0.44030135, -0.43790696, -0.43551257],
       [-0.5204607 , -0.51653326, -0.51260581, -0.50867836, -0.50475092,
        -0.50082347, -0.49689603, -0.49296858, -0.48904113, -0.48511369,
        -0.48118624, -0.4772588 , -0.47333135, -0.46940391, -0.46547646,
        -0.46154901, -0.45762157, -0.45369412, -0.44976668, -0.44583923,
        -0.44191178, -0.43798434, -0.43405689, -0.43012945, -0.426202  ,
        -0.42227455, -0.41834711, -0.41441966, -0.41049222, -0.40656477],
       [-0.54445061, -0.53772041, -0.5309902 , -0.52426   , -0.5175298 ,
        -0.51079959, -0.50406939, -0.49733919, -0.49060898, -0.48387878,
        -0.47714858, -0.47041837, -0.46368817, -0.45695797, -0.45022776,
        -0.44349756, -0.43676736, -0.43003715, -0.42330695, -0.41657675,
        -0.40984654, -0.40311634, -0.39638614, -0.38965594, -0.38292573,
        -0.37619553, -0.36946533, -0.36273512, -0.35600492, -0.34927472],
       [-0.56844052, -0.55890756, -0.5493746 , -0.53984164, -0.53030868,
        -0.52077571, -0.51124275, -0.50170979, -0.49217683, -0.48264387,
        -0.47311091, -0.46357795, -0.45404499, -0.44451203, -0.43497907,
        -0.42544611, -0.41591315, -0.40638019, -0.39684723, -0.38731427,
        -0.37778131, -0.36824834, -0.35871538, -0.34918242, -0.33964946,
        -0.3301165 , -0.32058354, -0.31105058, -0.30151762, -0.29198466],
       [-0.59243043, -0.58009471, -0.56775899, -0.55542327, -0.54308755,
        -0.53075184, -0.51841612, -0.5060804 , -0.49374468, -0.48140896,
        -0.46907325, -0.45673753, -0.44440181, -0.43206609, -0.41973037,
        -0.40739466, -0.39505894, -0.38272322, -0.3703875 , -0.35805178,
        -0.34571607, -0.33338035, -0.32104463, -0.30870891, -0.29637319,
        -0.28403748, -0.27170176, -0.25936604, -0.24703032, -0.2346946 ],
       [-0.61642033, -0.60128186, -0.58614338, -0.57100491, -0.55586643,
        -0.54072796, -0.52558948, -0.51045101, -0.49531253, -0.48017406,
        -0.46503558, -0.4498971 , -0.43475863, -0.41962015, -0.40448168,
        -0.3893432 , -0.37420473, -0.35906625, -0.34392778, -0.3287893 ,
        -0.31365083, -0.29851235, -0.28337388, -0.2682354 , -0.25309693,
        -0.23795845, -0.22281997, -0.2076815 , -0.19254302, -0.17740455],
       [-0.64041024, -0.62246901, -0.60452778, -0.58658654, -0.56864531,
        -0.55070408, -0.53276285, -0.51482161, -0.49688038, -0.47893915,
        -0.46099791, -0.44305668, -0.42511545, -0.40717422, -0.38923298,
        -0.37129175, -0.35335052, -0.33540929, -0.31746805, -0.29952682,
        -0.28158559, -0.26364435, -0.24570312, -0.22776189, -0.20982066,
        -0.19187942, -0.17393819, -0.15599696, -0.13805573, -0.12011449],
       [-0.66440015, -0.64365616, -0.62291217, -0.60216818, -0.58142419,
        -0.5606802 , -0.53993621, -0.51919222, -0.49844823, -0.47770424,
        -0.45696025, -0.43621626, -0.41547227, -0.39472828, -0.37398429,
        -0.3532403 , -0.33249631, -0.31175232, -0.29100833, -0.27026434,
        -0.24952035, -0.22877636, -0.20803237, -0.18728838, -0.16654439,
        -0.1458004 , -0.12505641, -0.10431242, -0.08356843, -0.06282444],
       [-0.68839006, -0.66484331, -0.64129656, -0.61774981, -0.59420307,
        -0.57065632, -0.54710957, -0.52356283, -0.50001608, -0.47646933,
        -0.45292258, -0.42937584, -0.40582909, -0.38228234, -0.35873559,
        -0.33518885, -0.3116421 , -0.28809535, -0.2645486 , -0.24100186,
        -0.21745511, -0.19390836, -0.17036161, -0.14681487, -0.12326812,
        -0.09972137, -0.07617462, -0.05262788, -0.02908113, -0.00553438],
       [-0.7123

# Broadcasting and automatic alignment

Labels help with combining arrays with different coordinates:

In [15]:
a = xr.DataArray(
    np.random.randn(3, 4),
    dims=("x", "y"),
    coords={"x": ["a", "b", "c"], "y": np.arange(4)},
)
b = xr.DataArray(
    np.random.randn(2, 7),
    dims=("x", "y"),
    coords={"x": ["b", "d"], "y": [-2, -1, 0, 1, 2, 3, 4]},
)

a + b

<xarray.DataArray (x: 1, y: 4)>
array([[ 0.57976778, -1.08659103, -2.5009955 , -0.11606741]])
Coordinates:
  * x        (x) object 'b'
  * y        (y) int64 0 1 2 3

This will automatically select only common labels from both arrays (a inner join) and then perform the operation.

Broadcasting works similar:

In [16]:
arr1 = xr.DataArray(
    np.random.randn(3),
    dims="x",
    coords={"x": ["a", "b", "c"]},
)
arr2 = xr.DataArray(
    np.random.randn(4),
    dims="y",
    coords={"y": np.arange(4)},
)

arr1 + arr2

<xarray.DataArray (x: 3, y: 4)>
array([[ 2.2332313 ,  0.77560262,  2.40122464,  1.38705455],
       [ 0.87452689, -0.58310178,  1.04252023,  0.02835014],
       [-0.01630089, -1.47392957,  0.15169245, -0.86247764]])
Coordinates:
  * x        (x) <U1 'a' 'b' 'c'
  * y        (y) int64 0 1 2 3

where both arrays were automatically broadcasted against each other:

In [17]:
arr1_, arr2_ = xr.broadcast(arr1, arr2)

In [18]:
arr1_

<xarray.DataArray (x: 3, y: 4)>
array([[ 1.17877957,  1.17877957,  1.17877957,  1.17877957],
       [-0.17992484, -0.17992484, -0.17992484, -0.17992484],
       [-1.07075262, -1.07075262, -1.07075262, -1.07075262]])
Coordinates:
  * x        (x) <U1 'a' 'b' 'c'
  * y        (y) int64 0 1 2 3

In [19]:
arr2_

<xarray.DataArray (x: 3, y: 4)>
array([[ 1.05445173, -0.40317695,  1.22244507,  0.20827498],
       [ 1.05445173, -0.40317695,  1.22244507,  0.20827498],
       [ 1.05445173, -0.40317695,  1.22244507,  0.20827498]])
Coordinates:
  * y        (y) int64 0 1 2 3
  * x        (x) <U1 'a' 'b' 'c'

and then the operation (a sum) was executed.